In [26]:
import logging
import os

import sqlite3
import pandas as pd

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger.setLevel('INFO')

db_file = './messages.db'

conn = sqlite3.connect(db_file, check_same_thread=True)

def run_sql(sql_str, db_con=conn, cols = None):
    with db_con as con:
        res = pd.DataFrame(con.execute(sql_str).fetchall(), columns=cols)
    return res
        
sql_str = """
    SELECT 
        name
    FROM 
        sqlite_master
    WHERE 
        name NOT LIKE 'sqlite_%';
"""
run_sql(sql_str, cols=['table_name'])

,table_name
0,tg_messages


In [27]:
TABLE_NAME = 'tg_messages'

table_df = pd.read_sql_query(f"SELECT * from {TABLE_NAME} LIMIT 10", conn)

table_df.head()

,id,msg,channel,msg_hash
0,2248,"управляющая компания предлагает в аренду 1 односпальный небольшой лофт после капитального ремонта с новой техникой и мебелью, район меса гитонья, недалеко от спиру киприану. граунд флор с двориком и парковкой. владелец запрашивает €1250 с контрактом на 1 год или €1100 с контрактом на 2 года. в стоимость включён интернет, вода и common expenses.",rentinlimassol,314c320246f78c0db8ddb8489072f7ab
1,2246,"управляющая компания предлагает в аренду целое жилое здание состоящее из пяти 2 спальных квартир в районе гермасойя, лимассол. здание будет сдаваться в аренду только юридическому лицу (не физическому ). все апартаменты полностью меблированы и оборудованное всеми необходимыми электроприборами. доступны с 1 сентября 2022 года. владелец запрашивает €13,500.",rentinlimassol,6f3d312ef2c7f712ba0f761556492df0
2,2238,"управляющая компания предлагает в аренду роскошную 5 спальную виллу в туристическом районе гермасоя. вилла расположена в закрытом и тихом комплексе, в пешей доступности до моря и всех удобств. комплекс состоит из 7 вилл и одного квартирного дома с общим бассейном. в доме 3 спальни на втором этаже и 2 дополнительные спальни на нижнем уровне, 1 гостевой туалет, 1 ванная комната, 3 душевых, система безопасности, видеодомофон, полы с подогревом и кондиционеры во всем доме, москитные сетки, крытая веранда, сад, крыша. сад, крытая парковка, большой подвал, который можно использовать как домашний кинотеатр или тренажерный зал. дом сдается полностью меблированным. владелец запрашивает €6000 в месяц.",rentinlimassol,cb3cb9c0da0195b069392b849003d84f
3,2232,"управляющая компания предлагает в аренду современную 1 спальную квартиру через дорогу от пляжа дасуди в районе гермасоя. квартира с хорошим ремонтом, со всей необходимой техникой и мебелью. большая крытая веранда. владелец запрашивает €1600 евро, контракт на два года.",rentinlimassol,3f29b9e54210ea6c4ea3cafb27d04a9b
4,2224,"управляющая компания предлагает в аренду роскошную 2 спальную квартиру с видом ну море в марине лимассола. есть своя парковка и кладовая комната. квартира полностью меблирована и оборудована брендовой техникой, vrv кондиционирование, большая крытая веранда. владелец запрашивает €5500 в месяц.",rentinlimassol,2c697381b6caac91e7b684fa34b86025


In [28]:
sql_str = f"""
    SELECT
        COUNT(*) as num_messages,
        CAST(AVG(length(msg))  as integer) as avg_length,
        COUNT(DISTINCT channel) num_channels
    FROM {TABLE_NAME}
    LIMIT 10
"""

pd.read_sql_query(sql_str, conn)

,num_messages,avg_length,num_channels
0,27037,324,6


In [29]:
from jinja2 import Template

def get_neg_samples_df():
    irrelevant_msg_ids = [
        153375, 130177, 152303, 156005, 152225, 152209, 152159, 152129,
        152831, 152766, 152740, 152697, 129161, 129139, 152628, 152556
    ]

    sql_str = Template(
        """
        SELECT 
            id, msg
        FROM {{ table }}
        WHERE id IN (
            {%- for msg_id in msg_ids -%} {{msg_id}} {{"," if not loop.last }} {% endfor %}
        )
        """
    ).render(msg_ids=irrelevant_msg_ids, table=TABLE_NAME)

    neg_samples_df = pd.read_sql_query(sql_str, conn)
    num_neg_samples = neg_samples_df.shape[0]

    logger.info('num rows: %d', num_neg_samples)
    
    return neg_samples_df

neg_samples_df = get_neg_samples_df()
neg_samples_df.head()

2023-01-03 16:12:44,486 num rows: 16


,id,msg
0,129161,"продажа (собственник) лимассол 580 000 евро urban city residences, a 201. вторичная продажа •количество спален: 3 •количество ванных: 2 •этаж: 2 этаж 5 этажного дома •общая крытая площадь: 121 м² •парковка: 1 •год постройки: 2021 полностью меблированная и укомплектованная всей необходимой техникой новая просторная 3-спальная квартира в современном комплексе, состоящем из 3-х пятиэтажных многоквартирных зданий, расположенном в центре лимассола, недалеко от ajax hotel. всего лишь 5-10 минут ходьбы отделяют вас от супермаркетов, магазинов, ресторанов, кафе, банков и множества других объектов городской инфраструктуры, и 15 - 20 минут - от пляжа. просторные жилые площади, красивые виды с веранд и удобное расположение -то, что позволяет urban city residences занимать достойное место среди недвижимости премиум класса. расположение в комплексе всего 30 апартаментов с большими верандами, отдельными кладовыми и парковочными местами, а также развитой внутренней инфраструктуры, включающей ..."
1,129139,аренда город ларнака . район декелия . дом три спальни . от 1350 в месяц плюс электричество и вода . свободен с 2 сентября. надежда 97638508
2,152740,"продажа продается новая квартира за €310,000 лимассол, закаки второй этаж 202 три спальни (2+1b) две ванные красивая веранда с хорошим видом личное парковочное место, кладовая энергоэффективность категория a солнечные панели для электроэнергии площадь квартиры 93.6m² крытая веранда 18.8m² цена квартиры €310,000 +vat квартира находится в хорошем районе 4 минуты от mymall 5 минут от limassol new marina 5 минут от самого большого казино и курорта в европе для консультации, и по всем вопросам звоните +357 9595 8898 y.a.f. estia real estate limited (reg: 1155/lic: 568/e) id 7.5"
3,152129,**mandarin park: новое высотное здание в лимассоле **в районе гермасойя возведут элитный небоскреб. [https://dom.com.cy/live/digest-63766](https://dom.com.cy/live/digest-63766/?utm_source=telegram)
4,152159,**как получить визитерскую визу на кипре? инструкция и новые правила **https://youtu.be/onnxf5ufbey


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer


def get_train_set(limit = -1):
    negatives_df = get_neg_samples_df()
    if limit < 0:
        num_negatives = negatives_df.shape[0]
        num_positives = int( num_negatives / 0.1)
        limit = num_positives + num_negatives
    else:
        limit = 8*10**3
    irrelevant_msg_ids = negatives_df['id'].values.tolist()
    sql_str = Template(
        """
        SELECT 
            msg,
            length(msg) as len_msg,
            CASE
                WHEN id IN (
                        {%- for msg_id in msg_ids -%} {{msg_id}} {{"," if not loop.last }} {% endfor %}
                    )
                THEN 1
                ELSE 0
            END target
        FROM {{ table }}
        ORDER BY target DESC
        LIMIT {{ limit }}
        """
    ).render(msg_ids=irrelevant_msg_ids, table=TABLE_NAME, limit=limit)

    corpus_df = pd.read_sql_query(sql_str, conn)
    
    return corpus_df

class Pandas2CSR:
    def __init__(self):
        self.vectorizer = None
        self.txt_col = None
        self.anchor_elements = None
    
    def df_to_matrix(self, input_series):
        res = input_series.values.reshape(-1).tolist()
        
        return res
    
    def fit(self, input_df, text_column='msg'):
        csr_matrix_dataset = self.df_to_matrix(input_df[text_column])
        self.txt_col = text_column
        
        logger.info('num rows: %d', len(csr_matrix_dataset))

        self.vectorizer = TfidfVectorizer()
        X = self.vectorizer.fit_transform(csr_matrix_dataset)
        logger.info('sparse matrix %s', X.shape)
        
        return X
    
    def transform(self, input_df):
        corpus = self.df_to_matrix(input_df[self.txt_col])
        X = self.vectorizer.transform(corpus)
        
        logger.info('result matrix %s', X.shape)
        
        return X
    
    def generate_features(self, neg_samples_df):
        # сохраняем якорные элементы
        if self.anchor_elements is None:
            self.anchor_elements = self.transform(neg_samples_df)
        anchor_elems

corpus_df = get_train_set()
pandas2csr = Pandas2CSR()
raw_matrix = pandas2csr.fit(corpus_df)

corpus_df.head()


2023-01-03 16:12:44,695 num rows: 16
2023-01-03 16:12:44,783 num rows: 176
2023-01-03 16:12:44,823 sparse matrix (176, 2390)


,msg,len_msg,target
0,"продажа (собственник) лимассол 580 000 евро urban city residences, a 201. вторичная продажа •количество спален: 3 •количество ванных: 2 •этаж: 2 этаж 5 этажного дома •общая крытая площадь: 121 м² •парковка: 1 •год постройки: 2021 полностью меблированная и укомплектованная всей необходимой техникой новая просторная 3-спальная квартира в современном комплексе, состоящем из 3-х пятиэтажных многоквартирных зданий, расположенном в центре лимассола, недалеко от ajax hotel. всего лишь 5-10 минут ходьбы отделяют вас от супермаркетов, магазинов, ресторанов, кафе, банков и множества других объектов городской инфраструктуры, и 15 - 20 минут - от пляжа. просторные жилые площади, красивые виды с веранд и удобное расположение -то, что позволяет urban city residences занимать достойное место среди недвижимости премиум класса. расположение в комплексе всего 30 апартаментов с большими верандами, отдельными кладовыми и парковочными местами, а также развитой внутренней инфраструктуры, включающей ...",2009,1
1,аренда город ларнака . район декелия . дом три спальни . от 1350 в месяц плюс электричество и вода . свободен с 2 сентября. надежда 97638508,140,1
2,"продажа продается новая квартира за €310,000 лимассол, закаки второй этаж 202 три спальни (2+1b) две ванные красивая веранда с хорошим видом личное парковочное место, кладовая энергоэффективность категория a солнечные панели для электроэнергии площадь квартиры 93.6m² крытая веранда 18.8m² цена квартиры €310,000 +vat квартира находится в хорошем районе 4 минуты от mymall 5 минут от limassol new marina 5 минут от самого большого казино и курорта в европе для консультации, и по всем вопросам звоните +357 9595 8898 y.a.f. estia real estate limited (reg: 1155/lic: 568/e) id 7.5",588,1
3,**mandarin park: новое высотное здание в лимассоле **в районе гермасойя возведут элитный небоскреб. [https://dom.com.cy/live/digest-63766](https://dom.com.cy/live/digest-63766/?utm_source=telegram),197,1
4,**как получить визитерскую визу на кипре? инструкция и новые правила **https://youtu.be/onnxf5ufbey,99,1


In [31]:
from sklearn.metrics.pairwise import euclidean_distances

neg_samples_csr = pandas2csr.transform(neg_samples_df)

distances = euclidean_distances(raw_matrix, neg_samples_csr)
logger.info(distances.shape)

2023-01-03 16:12:44,927 result matrix (16, 2390)
2023-01-03 16:12:44,937 (176, 16)


In [32]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(distances, corpus_df['target'])

In [33]:
corpus_df = get_train_set(limit=8*10**3)
raw_matrix = pandas2csr.transform(corpus_df)
distances = euclidean_distances(raw_matrix, neg_samples_csr)
neg_example_proba = lr.predict_proba(distances)

2023-01-03 16:12:45,199 num rows: 16
2023-01-03 16:12:45,625 result matrix (8000, 2390)


In [34]:
corpus_df['dummy_label'] = neg_example_proba[:,1]

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.expand_frame_repr', False)

scored_corpus_df = (
    corpus_df.query("len_msg > 0")
    [['msg', 'dummy_label']]
    .sort_values(by='dummy_label', ascending=False)
)

scored_corpus_df.head(270)

,msg,dummy_label
6486,ахахахахаха,0.987632
7005,halo,0.987632
7643,έχω μια επαγγελματική πρόταση και είναι επείγουσα,0.987632
4266,вот я,0.987632
7070,"вообще да, получали люди.",0.987632
...,...,...
269,"кипр, встречай **интернет-магазин epl diamond**! лучшие ювелирные изделия всемирного бренда теперь доступны к заказу он-лайн. покупайте ювелирные украшения на сайте [cy.epldiamond.com](http://cy.epldiamond.com/?utm_source=tgkipr_arenda) и получайте подарки! приятные цены и летние скидки вас порадуют! доставка по городу лимассол бесплатная, по кипру бесплатная при сумме заказа от 300 евро.",0.146483
4894,"******** ** [​](https://telegra.ph/file/0948a27872490e8a4a268.jpg)**#продажаземельногоучастка** **продаж** **зем**е**льного учаска** ———————— **#продажа**** **[**участка 12 000 ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)**[м²](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** **[**limassol](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)******[**agios tychonas ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** **стоимость 1**.**500 000 € ** **общая информация:** **обьект **№ **id: 0101 • mika** **•** **• **продаётся земельный участок с шикарным панорамным видом на море и город ,под инвестицию , в **агиос тихонас в **семи минутах от хайвей. **• **общая площадь [12 000 м²](https://telegra.ph/file/aece5c1a241defa34fa95.jpg) **• **площадь застройки [**5](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** % **• **дорога подведена к участку,а также рядом проведена электросеть,вся инфраструктура рядом развита. эти участок может быть интересен тем кто желает ин...",0.146131
4625,"[​](https://telegra.ph/file/8388ab0aff36200470150.jpg)[​](https://telegra.ph/file/89c0136e2c0d406fe7156.jpg)**#аренда** ****** **аренд** **пентхауса ** ———————— **аренда 3-х спального современного пентхауса **[**limassol](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)******[**potamos germasoia ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)** **возле гостинцы аполлония стоимость: 5.500 €** **( два депозит одна аренда при оплате вперед стоимость будет снижена до 5**.**000 € )** **общая информация:** **обьект **№ **id: 0305** **•sa•** **• современный пентхаус в совершено новом красивом доме,на последнем этаже, в 150-200 метрах от моря. ** [**описание ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)****** **• общая крытая площадь **[**132 ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)**[м²](https://telegra.ph/file/aece5c1a241defa34fa95.jpg) **• kрытые веранды **[**39 ](https://telegra.ph/file/aece5c1a241defa34fa95.jpg)**[м²](https://telegra.p...",0.145923
53,**yoo limassol by philippe starck: эксклюзивный жилой комплекс в лимассоле** компания-застройщик презентовала свой новый проект. (фото) https://dom.com.cy/live/digest-56818/,0.145286


In [35]:
scored_corpus_df.to_csv('./scored_corpus.csv', index=False)
logger.info('%d lines saved', scored_corpus_df.shape[0])

2023-01-03 16:12:45,910 6542 lines saved


In [36]:
df = pd.read_csv('./labeled_data_corpus.csv')

train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print(train_df.shape[0], train_df['label'].mean(), test_df.shape[0], test_df['label'].mean())

df.head()

KeyError: 'subset'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import numpy as np
from sklearn.model_selection import ParameterGrid

X_train = train_df['msg'].values
y_train = train_df['label']

X_test = test_df['msg'].values
y_true = test_df['label']

grid = {
    'max_df': np.arange(0.01, 1.0, 0.01),
    'min_df': np.arange(1, 20, 1),
}

best_params = {'max_df': None, 'min_df': None}
best_score = 0.0

num_iters = len(ParameterGrid(grid))

cnt = 0
for param_values in ParameterGrid(grid):
    # fit 
    vectorizer = TfidfVectorizer(**param_values).fit(X_train)
    X_train_csr = vectorizer.transform(X_train)
    lr = LogisticRegression().fit(X_train_csr, y_train)
    # predict
    X_test_csr = vectorizer.transform(X_test)
    y_pred = lr.predict(X_test_csr)
    cur_score = f1_score(y_true, y_pred)
    if cur_score > best_score:
        best_score = cur_score
        best_params.update(param_values)
    if cnt % 250 == 0:
        logging.info(
            'iteration: %d of %d; %s; best_score= %.4f',
            cnt, num_iters, param_values, best_score
        )
    cnt = cnt + 1
print('best params: %s, best_score %.4f', best_params, best_score)

2023-01-01 22:01:40,603 iteration: 0 of 1881; {'max_df': 0.01, 'min_df': 1}; best_score= 0.6173
2023-01-01 22:04:01,478 iteration: 250 of 1881; {'max_df': 0.14, 'min_df': 4}; best_score= 0.8549
2023-01-01 22:06:25,944 iteration: 500 of 1881; {'max_df': 0.27, 'min_df': 7}; best_score= 0.8611
2023-01-01 22:08:56,314 iteration: 750 of 1881; {'max_df': 0.4, 'min_df': 10}; best_score= 0.8611
2023-01-01 22:11:20,836 iteration: 1000 of 1881; {'max_df': 0.53, 'min_df': 13}; best_score= 0.8611
2023-01-01 22:13:35,105 iteration: 1250 of 1881; {'max_df': 0.66, 'min_df': 16}; best_score= 0.8611
2023-01-01 22:15:44,804 iteration: 1500 of 1881; {'max_df': 0.79, 'min_df': 19}; best_score= 0.8611
2023-01-01 22:17:55,489 iteration: 1750 of 1881; {'max_df': 0.93, 'min_df': 3}; best_score= 0.8611


best params: %s, best_score %.4f {'max_df': 0.2, 'min_df': 18} 0.8610567514677103


In [ ]:
df.reset_index().rename(columns={'index': 'msg_id'}).head()

,msg_id,msg_id,msg,label,subset
0,0,0,"здравствуйте. ишу 2х спальную квартиру в лимассоле. желательно гермасойя. семья из 2х взрослых и 2х детей. без животных. на длительный срок, бюджет до 1000-1500 евро. предложения в лс.",0,train
1,1,1,#сниму комнату в лимассоле или недалеко от него. с начала августа. любые предложения в лс,0,train
2,2,2,мошенник риэлторским услугам.,0,train
3,3,3,"**sales** reg.1053 lic.489/e **stylish apartment with sea view kissonerga. paphos** •total area: 85 m2 + balcony •bedrooms: 2 •bathrooms: 1 **€ 120,000** we have a lot to offer ================ **продажа** reg.1053 lic.489/e **стильные апартаменты вид на море •••kissonerga. пафос. ** •общая площадь: 85м2 + балкон •спальни: 2 •ванные комнаты: 1 **€ 120 000 ****+35726935826**** директ telegram 24/7** у нас есть что вам предложить",0,train
4,4,4,"важно: [valerii korol](tg://user?id=193474890), если ты не бот и не спамер, пройди проверку, нажав на кнопку, где есть",0,train


In [ ]:
df.reset_index().rename(columns={'index': 'msg_id'}).to_csv('./labeled_data_corpus.csv', index=False)

In [ ]:
df = pd.read_csv('./labeled_data_corpus.csv')[['msg', 'label']]

train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print(train_df.shape[0], train_df['label'].mean(), test_df.shape[0], test_df['label'].mean())

KeyError: 'subset'